# 📌 애니메이션 목록 불러오기

In [ ]:
import dotenv
import os
import pymongo
import requests
import json

In [ ]:
# 애니메이션 정보 가져오기
def get_animation():
  # request url
  laftel_api = "https://laftel.net/api/search/v1/discover/"

  # required header
  header = {"laftel": "TeJava"}
  
  # 애니메이션 목록 저장 리스트
  ani_list = []
  
  # next가 없을 때까지 api 호출
  while(True):
    response = requests.get(url=laftel_api, headers=header)
    
    # 가져온 결과 저장
    result = response.json()
    
    # 가져온 애니메이션 총 수
    count = result["count"]
    
    # 애니메이션 상세 정보
    results = result["results"]
    
    # 애니메이션 목록 저장
    ani_list.append(results)  
    
    # 다음 정보 가져오는 api
    next = result["next"]
    
    # 다음 가져올 데이터가 있다면 api 호출
    if (next is not None):
      laftel_api = next
    else:
      break
  
  return ani_list

ani_list = get_animation()

print("애니메이션 목록 불러오기 완료")

In [ ]:
# 애니메이션 정보 DB에 저장
def save_animation(data):
        
    # 환경변수 불러오기
    dotenv.load_dotenv(dotenv.find_dotenv())
    USER = os.environ["MONGODB_USER"] # MongoDB user
    PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
    PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port
    
    # DB 연결
    #   conn = pymongo.MongoClient()
    conn = pymongo.MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)
    db = conn.test
    dbcol_ani = db.animation
  
  # data: [[20개의 애니상세목록], [20개의 애니상세 목록]...]
    for unit in data: # unit: 20개의 애니 상세 목록
        for u in unit: # u: 애니 상세 정보
#             dbcol_ani.insert_one(u) # DB에 삽입
            dbcol_ani.update_one({"id": u["id"]}, {"$set": u }, upsert = True) # 있으면 수정, 없으면 삽입

save_animation(ani_list)

print("애니메이션 정보 저장 완료")